In [2]:
from os import sys
# Path to workspace
sys.path.insert(0, '/workspace/dense-self-supervised-representation-learning-for-3D-shapes/')

import h5py
import torch
import numpy as np
from tqdm import tqdm
import k3d

In [3]:
import neptune.new as neptune
from workspace.utils.train_loop import *

params = {
    'name': 'dgcnn_normals_modelnet_supervised',
    'dataset': 'modelnet',
    'batch_size': 30,
    'tau': 0.07,
    'n_output': 512,
    'result_dim': 3,
    'hidden_dim': 256,
    'total_epochs': 100,
    'lr': 0.001,
    'weight_decay': 1e-5,
    'save_every': 50,
    'weights_root': 'weights/'
}

# tags
tags = ['modelnet', 'normals', 'supervised']

logger = neptune.init(
    project="igor3661/crossmodal",
    name=params['name'],
    tags=tags,
    api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcG'\
              'lfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiN'\
              'zcxMGNkOS04ZjU3LTRmNDMtOWFjMS1kNDNkZDZlNDI4YWYifQ==',
)  # your credentials


logger['parameters'] = params

device = 'cuda:3'

https://app.neptune.ai/igor3661/crossmodal/e/CROSS-60


/opt/conda/lib/python3.8/site-packages/neptune/new/internal/utils/git.py:35: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")
/opt/conda/lib/python3.8/site-packages/neptune/new/internal/utils/git.py:35: UserWarning: GitPython could not be initialized
  warnings.warn("GitPython could not be initialized")


Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [5]:
from torch.utils.data import Dataset, DataLoader

class NormalsDataset(Dataset):
    def __init__(self, data_path, transform=None):
        super().__init__()
        self.transform = transform
        self.file = h5py.File(data_path, 'r')

    def __getitem__(self, index):        
        points = self.file['points'][index][:]
        normals = self.file['point_normals'][index][:]

        if self.transform is not None:
            points = self.transform(points)

        points = torch.from_numpy(points).float()
        points = torch.permute(points, (1, 0))
        normals = torch.from_numpy(normals).float()
        normals = torch.permute(normals, (1, 0))
        return points, normals

        
    def __len__(self):
        return self.file['points'].shape[0]

In [6]:
from workspace.crossmodal.utils.collates import move_to_device, sample
from torch.utils.data import default_collate


train_data = NormalsDataset('modelnet/modelnet_train_1024.h5')
test_data = NormalsDataset('modelnet/modelnet_test_1024.h5')


train_loader = DataLoader(
    train_data,
    shuffle=True,
    batch_size=params['batch_size'],
    num_workers=5,
)
test_loader = DataLoader(
    test_data,
    shuffle=False,
    batch_size=params['batch_size'],
    num_workers=5
)

In [11]:
class Transpose(torch.nn.Module):
    def __init__(self, *dims):
        super().__init__()
        self.dims = dims

    def forward(self, data):
        return data.transpose(*self.dims)
    

class NormalsModel(torch.nn.Module):
    def __init__(self, model, model_output_dim, result_dim, hidden_dim):
        super().__init__()
        self.model = model
        self.head = torch.nn.Sequential(
            Transpose(1, 2),
            torch.nn.Linear(model_output_dim, hidden_dim),
            Transpose(1, 2),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            Transpose(1, 2),
            torch.nn.Linear(hidden_dim, 128),
            Transpose(1, 2),
        )
        
    def forward(self, data):
        return self.head(self.model.forward_features(data))

In [14]:
from workspace.models.dgcnn import DGCNN
dgcnn = DGCNN(n_patches=5)

model = NormalsModel(
    dgcnn,
    model_output_dim=params['n_output'],
    hidden_dim=params['hidden_dim'],
    result_dim=params['result_dim']
).to(device)

model.load_state_dict(torch.load('weights/CROSS-56/100epoch.pt'))

head = torch.nn.Sequential(
    Transpose(1, 2),
    torch.nn.Linear(params['n_output'], params['hidden_dim']),
    Transpose(1, 2),
    torch.nn.BatchNorm1d(params['hidden_dim']),
    torch.nn.ReLU(),
    Transpose(1, 2),
    torch.nn.Linear(params['hidden_dim'], 3),
    Transpose(1, 2),
)

model.head = head.to(device)

In [15]:
from torch.nn import functional as F

def normals_loss(outs, normals):
    outs = F.normalize(outs, dim=1)
    
    return (1 - (outs * normals).sum(1) ** 2).mean()

def forward( 
    model,
    batch, # raw data from dataloader
    logger, # neptune run
    mode # 'train'/'val'
): # -> loss

    points, normals = move_to_device(batch, device)
    outs = model(points)
    
    loss = normals_loss(outs, normals)
    
    return {
        'loss': loss
    }

In [16]:
def get_warmup_schedule(
        optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, last_epoch=-1
):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        progress = float(current_step - num_warmup_steps) /\
                   float(max(1, num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + cos(pi * float(num_cycles) * 2.0 * progress)))

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch)

optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=params['lr'],
    weight_decay=params['weight_decay']
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_loader) * params['total_epochs'])

#scheduler = get_warmup_schedule(optimizer, 4 * len(train_loader), params['total_epochs'] * len(train_loader))

In [17]:
train_model(model, params, logger,  train_loader, test_loader, optimizer, scheduler, forward)

Validation: 100%|██████████| 83/83 [00:16<00:00,  5.01it/s, Loss=0.0428]


Validation: 100%|██████████| 83/83 [00:17<00:00,  4.87it/s, Loss=0.0392]
